# Optimizing for CO2 storage from CO2 Emitters

## Setup

In [1]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance
from geopy.distance import great_circle
import random
import pickle

In [2]:
def facility_cleaning(fac_df, emiss_df):
    facilitiesB = fac_df.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})

    emsCA2019loc = pd.merge(emiss_df, facilitiesB[['Facility Id',
                                                    'facility_name',
    #                                               'Facility Name2',
                                                   'Latitude',
                                                   'Longitude',
                                                   'supply_metric',
    #                                               'Primary NAICS Code2',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
    emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'supply_metric']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'supply_metric':'sum'})
    emsCA2019locagg.reset_index(inplace=True)
    emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon'})
    emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['supply_metric'] > 0]
    facility_latlon = emsCA2019locagg[['lat', 'lon']].to_records(index=False)
    facility_ids = emsCA2019locagg['facility_id'].tolist()
    facility_names = emsCA2019locagg['facility_name'].tolist()
    supply_volume = emsCA2019locagg['supply_metric'].tolist()
    supply_trucks = [i/25 for i in supply_volume]
    return facility_latlon, facility_ids, facility_names, supply_volume, supply_trucks


def get_distances(facility_latlon, pools_latlon):
    pools_distance_matrix = []
    for i in range(len(facility_latlon)):
        distances = []
        for j in range(len(pools_latlon)):
            dist = great_circle(facility_latlon[i], pools_latlon[j]).miles
            distances.append(dist)
        pools_distance_matrix.append(distances)
    return pools_distance_matrix


def get_complex_costs(distance_matrix, pool_trucks):
    complex_costs = []
    for i in range(len(distance_matrix)):
        new = []
        for j in range(len(distance_matrix[i])):
            opening_cost = 2000000/pool_trucks[j]
            # opening cost = $2,000,000/number of trucks for the pool
            # cost / truck = 2*$2*distance + opening cost
            cost = 4 * distance_matrix[i][j] + opening_cost
            new.append(cost)
        complex_costs.append(new)
    return complex_costs


def run_optimization(n_receivers, n_suppliers, supply_trucks, demand_trucks, costs, distances, facility_names, facility_latlon, pools_ids, pools_latlon, industry_str, year):
    print("Running optimization for {} industry and {} year timeframe".format(industry_str, year))
    supply_trucks = [i*year for i in supply_trucks]
    
    p_prob = LpProblem('Unbalaced_Transportation_Problem', LpMinimize)
    #costs = pools_distance_matrix_complexcost
    routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

    x = LpVariable.dicts('X', routes, lowBound=0)
    p_prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])

    for i in range(n_suppliers):
        p_prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply_trucks[i]

    for j in range(n_receivers):
        p_prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand_trucks[j]

    # Solving problem
    p_prob.solve()
    print("Problem successfully solved")
    
    volumes_moved = [i.varValue for i in p_prob.variables()]
    volume_matrix = []
    k=0
    while k < n_suppliers:
        vols = []
        for i in range(n_receivers):
            j = i + k*n_receivers
            vols.append(volumes_moved[j])
        volume_matrix.append(vols)
        k+=1
    print("Volume matrix created")
        
    results = []
    for i in range(n_suppliers):
        f_name = facility_names[i]
        f_loc = facility_latlon[i]
        for j in range(n_receivers):
            vol = volume_matrix[i][j]
            if vol > 0:
                w_id = pools_ids[j]
                w_loc = pools_latlon[j]
                dist = distances[i][j]
                cost = costs[i][j] * vol
                r = [f_name, f_loc, w_id, w_loc, dist, cost, vol, industry_str, year]
                results.append(r)
    results_df = pd.DataFrame(results, columns = ['facility_name', 'facility_location','pool_id','pool_location','distance','cost','n_trucks', 'industry', 'timeframe'])
    print("Results created")
    print()
    print("--------------------------------------------")
    
    return results_df


## Data import and cleaning

In [3]:
DATADIR = 'D:/W210_Capstone/co2-storage-fall2021/'
os.chdir(DATADIR)

wells = pd.read_csv("./data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_ids = wells_oilgas['API'].tolist()
pooled_wells = pd.read_csv("./data/pool_volumes.csv")
pools_latlon = pooled_wells[['Latitude', 'Longitude']].to_records(index=False)
pools_ids = pooled_wells['pool_id'].tolist()
pools_volumes = pooled_wells['totalco2'].tolist()
pools_trucks = [i/25 for i in pools_volumes]
n_pools = len(pools_ids)

In [4]:
emissions = pd.read_csv(r'./data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

## Prepare data for optimization

In [5]:
facilities = facilities[(facilities.State == 'CA')]
facilities['supply_metric'] = (facilities['Cement Production'] + facilities['Petroleum Refining'] +
                               facilities['Electricity Generation'] + facilities['Iron and Steel Production'])
facilities_cement = facilities[facilities['Cement Production'] > 0]
facilities_cement['supply_metric'] = facilities_cement['Cement Production']
facilities_oil = facilities[facilities['Petroleum Refining'] > 0]
facilities_oil['supply_metric'] = facilities_oil['Petroleum Refining']
facilities_energy = facilities[facilities['Electricity Generation'] > 0]
facilities_energy['supply_metric'] = facilities_energy['Electricity Generation']
facilities_steel = facilities[facilities['Iron and Steel Production'] > 0]
facilities_steel['supply_metric'] = facilities_steel['Iron and Steel Production']

facility_latlon, facility_ids, facility_names, supply_volume, supply_trucks = facility_cleaning(facilities, emsCA2019)
n_facilities = len(facility_ids)

facility_c_latlon, facility_c_ids, facility_c_names, supply_c_volume, supply_c_trucks = facility_cleaning(facilities_cement, emsCA2019)
n_facilities_c = len(facility_c_ids)

facility_o_latlon, facility_o_ids, facility_o_names, supply_o_volume, supply_o_trucks = facility_cleaning(facilities_oil, emsCA2019)
n_facilities_o = len(facility_o_ids)

facility_e_latlon, facility_e_ids, facility_e_names, supply_e_volume, supply_e_trucks = facility_cleaning(facilities_energy, emsCA2019)
n_facilities_e = len(facility_e_ids)

facility_s_latlon, facility_s_ids, facility_s_names, supply_s_volume, supply_s_trucks = facility_cleaning(facilities_steel, emsCA2019)
n_facilities_s = len(facility_s_ids)

facility_total_latlon = np.concatenate((facility_c_latlon, facility_o_latlon, facility_e_latlon, facility_s_latlon))
facility_total_ids = np.concatenate((facility_c_ids, facility_o_ids, facility_e_ids, facility_s_ids))
facility_total_names = np.concatenate((facility_c_names, facility_o_names, facility_e_names, facility_s_names))
supply_volume_total = np.concatenate((supply_c_volume, supply_o_volume, supply_e_volume, supply_s_volume))
supply_trucks_total = np.concatenate((supply_c_trucks, supply_o_trucks, supply_e_trucks, supply_s_trucks))
n_facilities_total = len(facility_total_ids)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [23]:
print(facilities_cement['supply_metric'].sum() * 10 / 1000000)
print(facilities_oil['supply_metric'].sum() * 10 / 1000000)
print(facilities_energy['supply_metric'].sum() * 10 / 1000000)
print(facilities_steel['supply_metric'].sum() * 10 / 1000000)

77.42378892
60.76586904
277.09608985999995
0.143029


In [6]:
facilitiesB = facilities_cement.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})
emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
                                                    'facility_name',
    #                                               'Facility Name2',
                                                   'city',
                                                   'Zip Code',
                                                   'Address',
                                                   'County',
                                                   'Latitude',
                                                   'Longitude',
    #                                               'Primary NAICS Code2',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon',
                                                      'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg

,facility_id,facility_name,lat,lon,annual_emissions
0,1002308,CEMEX Construction Materials Pacific LLC,34.622200,-117.100100,12834.3
1,1002431,HANSON PERMANENTE CEMENT,37.318100,-122.091000,464.9
2,1004612,LEHIGH SOUTHWEST CEMENT CO.,40.736900,-122.322300,1072.2
3,1005662,Mitsubishi Cement Corp Cushenbury Cement Plant,34.437557,-116.891034,421.7
4,1006642,NATIONAL CEMENT CO OF CALIFORNIA INC,34.819863,-118.748732,2476.8
5,1006842,CalPortland Company Mojave Plant,35.029298,-118.316236,712.0
6,1007927,CalPortland Company Oro Grande Plant,34.604500,-117.338200,270.5


In [7]:
supply_c_volume

[11370438.288,
 1541474.944,
 292140.03,
 1063161.868,
 1586353.484,
 1123385.34,
 1250724.64]

In [8]:
distances_c = get_distances(facility_c_latlon, pools_latlon)
complex_costs_c = get_complex_costs(distances_c, pools_trucks)

distances_o = get_distances(facility_o_latlon, pools_latlon)
complex_costs_o = get_complex_costs(distances_o, pools_trucks)

distances_e = get_distances(facility_e_latlon, pools_latlon)
complex_costs_e = get_complex_costs(distances_e, pools_trucks)

distances_s = get_distances(facility_s_latlon, pools_latlon)
complex_costs_s = get_complex_costs(distances_s, pools_trucks)

distances_total = get_distances(facility_total_latlon, pools_latlon)
complex_costs_total = get_complex_costs(distances_total, pools_trucks)

In [9]:
industry_list = ['cement', 'oil_refining', 'steel', 'energy', 'total']
year_list = [1, 5, 10, 20, 30]

inputs_list = []
for i in year_list:
    cement = [n_facilities_c, supply_c_trucks, complex_costs_c, distances_c, facility_c_names, facility_c_latlon, 'cement', i]
    oil = [n_facilities_o, supply_o_trucks, complex_costs_o, distances_o, facility_o_names, facility_o_latlon, 'oil_refining', i]
    energy = [n_facilities_e, supply_e_trucks, complex_costs_e, distances_e, facility_e_names, facility_e_latlon, 'energy', i]
    steel = [n_facilities_s, supply_s_trucks, complex_costs_s, distances_s, facility_s_names, facility_s_latlon, 'steel', i]
    total = [n_facilities_total, supply_trucks_total, complex_costs_total, distances_total, facility_total_names, facility_total_latlon, 'total', i]
    
    inputs_list.append(cement)
    inputs_list.append(oil)
    inputs_list.append(steel)
    inputs_list.append(energy)
    inputs_list.append(total)

    

## Run optimization and write results

In [10]:
detailed_results = pd.DataFrame()

for i in inputs_list:
    results = run_optimization(n_pools,
                               i[0],
                               i[1],
                               pools_trucks,
                               i[2],
                               i[3],
                               i[4],
                               i[5],
                               pools_ids,
                               pools_latlon,
                               i[6],
                               i[7])
    detailed_results = detailed_results.append(results)


Running optimization for cement industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for oil_refining industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for steel industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for energy industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for total industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for cement industry and 5 year timeframe
Problem successfully solved
Volume matrix created
Results create

In [11]:
# re calculate cost as 2 * ($2*miles)*(volume/25) + 2,000,000
detailed_results['volume'] = detailed_results['n_trucks']*25
detailed_results['cost_per_ton'] = detailed_results['cost']/detailed_results['volume']
detailed_results.sort_values(by=['cost_per_ton'])

,facility_name,facility_location,pool_id,pool_location,distance,cost,n_trucks,industry,timeframe,volume,cost_per_ton
8444,Salinas River Cogeneration Facility,"[35.9515, -120.8679]",San Ardo:Main:Lombardi,"[35.948221295, -120.8602182]",0.485730,1.061875e+04,1746.74110,energy,30,43668.52750,2.431671e-01
2485,TRICOR REFINING LLC,"[35.4185, -119.0112]",Kern River::Kern River,"[35.447206, -118.98935600000002]",2.333698,5.535377e+04,5528.76180,total,30,138219.04500,4.004786e-01
3193,TRICOR REFINING LLC,"[35.4185, -119.0112]",Kern River::Kern River,"[35.447206, -118.98935600000002]",2.333698,5.535377e+04,5528.76180,total,20,138219.04500,4.004786e-01
23877,"California Resources Elk Hills, LLC - Gas Proc...","[35.23893, -119.35951]",Elk Hills::Upper (Undifferentiated),"[35.2720925, -119.39971795]",3.224357,7.827294e+03,572.85550,total,10,14321.38750,5.465458e-01
17944,La Paloma Generating Plant,"[35.2956, -119.5919]",Elk Hills::Stevens (29R),"[35.295080525, -119.533703]",3.282099,5.978215e+03,433.67946,total,10,10841.98650,5.513948e-01
...,...,...,...,...,...,...,...,...,...,...,...
26061,El Centro,"[32.8022, -115.54]",Belridge North:pool,"[35.517082, -119.762591]",305.671777,3.258615e+09,107.00605,total,20,2675.15125,1.218105e+06
16738,Blythe Energy,"[33.6157, -114.6865]",Belridge North:pool,"[35.517082, -119.762591]",317.229328,3.258619e+09,107.00605,energy,20,2675.15125,1.218107e+06
9335,Redding Power Plant,"[40.5083, -122.4253]",Belridge North:pool,"[35.517082, -119.762591]",374.032053,3.258644e+09,107.00605,total,10,2675.15125,1.218116e+06
14702,Redding Power Plant,"[40.5083, -122.4253]",Belridge North:pool,"[35.517082, -119.762591]",374.032053,3.258644e+09,107.00605,total,20,2675.15125,1.218116e+06


In [12]:
summary_results = detailed_results.groupby(by=['industry','timeframe']).agg(
    volume_transferred=('volume', 'sum'),
    total_cost=('cost', 'sum'),
    total_distance=('distance', 'sum')  
)
summary_results['cost_per_ton'] = summary_results['total_cost']/summary_results['volume_transferred']
summary_results



volume_transferred    total_cost  total_distance  \
industry     timeframe                                                     
cement       1                1.822768e+07  7.160608e+09    1.913577e+03   
             5                9.113839e+07  3.575466e+10    4.070699e+03   
             10               1.822768e+08  4.518601e+10    4.603964e+03   
             20               3.645536e+08  7.842704e+12    1.703940e+04   
             30               5.468304e+08  7.897318e+12    2.590614e+04   
energy       1                1.051721e+08  5.572413e+12    1.816064e+04   
             5                5.258606e+08  7.845148e+12    3.082578e+04   
             10               1.051721e+09  8.188888e+12    6.343124e+04   
             20               2.376102e+09  1.195153e+13    5.812750e+06   
             30               4.525479e+09  1.583464e+13    5.835653e+06   
oil_refining 1                6.605261e+07  5.483917e+12    3.220189e+03   
             5                3.302631e+08  7.805848e+12    1.331890e+04   
             10               6.605261e+08  7.999460e+12    3.407838e+04   
             20               1.321052e+09  8.331118e+12    5.187033e+04   
             30               2.011688e+09  1.030495e+13    1.302614e+06   
steel        1                1.430290e+04  7.566748e+06    4.553892e+01   
             5                7.151450e+04  3.783374e+07    4.553892e+01   
             10               1.430290e+05  7.566748e+07    4.553892e+01   
             20               2.860580e+05  1.513350e+08    4.553892e+01   
             30               4.290870e+05  2.270024e+08    4.553892e+01   
total        1                1.894667e+08  7.735821e+12    2.711839e+04   
             5                9.473335e+08  8.139369e+12    6.787420e+04   
             10               2.126402e+09  9.838586e+12    5.331090e+06   
             20               4.858917e+09  1.458105e+13    6.672005e+06   
             30               7.327821e+09  2.365614e+13    5.777978e+06   

                        cost_per_ton  
industry     timeframe                
cement       1            392.842545  
             5            392.311689  
             10           247.897762  
             20         21513.173942  
             30         14441.988497  
energy       1          52983.752992  
             5          14918.683724  
             10          7786.178301  
             20          5029.890920  
             30          3498.997791  
oil_refining 1          83023.474591  
             5          23635.242031  
             10         12110.739163  
             20          6306.426028  
             30          5122.537094  
steel        1            529.035923  
             5            529.035923  
             10           529.035923  
             20           529.035923  
             30           529.035923  
total        1          40829.448257  
             5           8591.872294  
             10          4626.870965  
             20          3000.884747  
             30          3228.263429

In [13]:
detailed_results['volume'] = detailed_results['volume']/1000
detailed_results['cost'] = detailed_results['cost']/1000000

In [14]:
summary_results['volume_transferred'] = summary_results['volume_transferred']/1000
summary_results['total_cost'] = summary_results['total_cost']/1000000

In [15]:
detailed_results.sort_values(by=['cost_per_ton'])

,facility_name,facility_location,pool_id,pool_location,distance,cost,n_trucks,industry,timeframe,volume,cost_per_ton
8444,Salinas River Cogeneration Facility,"[35.9515, -120.8679]",San Ardo:Main:Lombardi,"[35.948221295, -120.8602182]",0.485730,0.010619,1746.74110,energy,30,43.668527,2.431671e-01
2485,TRICOR REFINING LLC,"[35.4185, -119.0112]",Kern River::Kern River,"[35.447206, -118.98935600000002]",2.333698,0.055354,5528.76180,total,30,138.219045,4.004786e-01
3193,TRICOR REFINING LLC,"[35.4185, -119.0112]",Kern River::Kern River,"[35.447206, -118.98935600000002]",2.333698,0.055354,5528.76180,total,20,138.219045,4.004786e-01
23877,"California Resources Elk Hills, LLC - Gas Proc...","[35.23893, -119.35951]",Elk Hills::Upper (Undifferentiated),"[35.2720925, -119.39971795]",3.224357,0.007827,572.85550,total,10,14.321388,5.465458e-01
17944,La Paloma Generating Plant,"[35.2956, -119.5919]",Elk Hills::Stevens (29R),"[35.295080525, -119.533703]",3.282099,0.005978,433.67946,total,10,10.841987,5.513948e-01
...,...,...,...,...,...,...,...,...,...,...,...
26061,El Centro,"[32.8022, -115.54]",Belridge North:pool,"[35.517082, -119.762591]",305.671777,3258.614549,107.00605,total,20,2.675151,1.218105e+06
16738,Blythe Energy,"[33.6157, -114.6865]",Belridge North:pool,"[35.517082, -119.762591]",317.229328,3258.619496,107.00605,energy,20,2.675151,1.218107e+06
9335,Redding Power Plant,"[40.5083, -122.4253]",Belridge North:pool,"[35.517082, -119.762591]",374.032053,3258.643809,107.00605,total,10,2.675151,1.218116e+06
14702,Redding Power Plant,"[40.5083, -122.4253]",Belridge North:pool,"[35.517082, -119.762591]",374.032053,3258.643809,107.00605,total,20,2.675151,1.218116e+06


In [16]:
list_results = detailed_results.values.tolist()
file_name = 'data/results/full_detailed_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_results, open_file)
open_file.close()

html_results = detailed_results.to_html()
html_file = open("data/results/full_detailed_results.html", "w")
html_file.write(html_results)
html_file.close()

In [17]:
total_pool_volume = sum(pools_volumes)/1000

summary_results['percent_pool_volume_used'] = summary_results['volume_transferred'] / total_pool_volume
summary_results

volume_transferred    total_cost  total_distance  \
industry     timeframe                                                     
cement       1                1.822768e+04  7.160608e+03    1.913577e+03   
             5                9.113839e+04  3.575466e+04    4.070699e+03   
             10               1.822768e+05  4.518601e+04    4.603964e+03   
             20               3.645536e+05  7.842704e+06    1.703940e+04   
             30               5.468304e+05  7.897318e+06    2.590614e+04   
energy       1                1.051721e+05  5.572413e+06    1.816064e+04   
             5                5.258606e+05  7.845148e+06    3.082578e+04   
             10               1.051721e+06  8.188888e+06    6.343124e+04   
             20               2.376102e+06  1.195153e+07    5.812750e+06   
             30               4.525479e+06  1.583464e+07    5.835653e+06   
oil_refining 1                6.605261e+04  5.483917e+06    3.220189e+03   
             5                3.302631e+05  7.805848e+06    1.331890e+04   
             10               6.605261e+05  7.999460e+06    3.407838e+04   
             20               1.321052e+06  8.331118e+06    5.187033e+04   
             30               2.011688e+06  1.030495e+07    1.302614e+06   
steel        1                1.430290e+01  7.566748e+00    4.553892e+01   
             5                7.151450e+01  3.783374e+01    4.553892e+01   
             10               1.430290e+02  7.566748e+01    4.553892e+01   
             20               2.860580e+02  1.513350e+02    4.553892e+01   
             30               4.290870e+02  2.270024e+02    4.553892e+01   
total        1                1.894667e+05  7.735821e+06    2.711839e+04   
             5                9.473335e+05  8.139369e+06    6.787420e+04   
             10               2.126402e+06  9.838586e+06    5.331090e+06   
             20               4.858917e+06  1.458105e+07    6.672005e+06   
             30               7.327821e+06  2.365614e+07    5.777978e+06   

                        cost_per_ton  percent_pool_volume_used  
industry     timeframe                                          
cement       1            392.842545                  0.010491  
             5            392.311689                  0.052457  
             10           247.897762                  0.104913  
             20         21513.173942                  0.209826  
             30         14441.988497                  0.314739  
energy       1          52983.752992                  0.060534  
             5          14918.683724                  0.302669  
             10          7786.178301                  0.605339  
             20          5029.890920                  1.367613  
             30          3498.997791                  2.604729  
oil_refining 1          83023.474591                  0.038018  
             5          23635.242031                  0.190089  
             10         12110.739163                  0.380179  
             20          6306.426028                  0.760358  
             30          5122.537094                  1.157867  
steel        1            529.035923                  0.000008  
             5            529.035923                  0.000041  
             10           529.035923                  0.000082  
             20           529.035923                  0.000165  
             30           529.035923                  0.000247  
total        1          40829.448257                  0.109051  
             5           8591.872294                  0.545257  
             10          4626.870965                  1.223893  
             20          3000.884747                  2.796646  
             30          3228.263429                  4.217672

In [18]:
list_summary_results = summary_results.values.tolist()
file_name = 'data/results/summary_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_summary_results, open_file)
open_file.close()

html_summary_results = detailed_results.to_html()
html_file = open("data/results/summary_results.html", "w")
html_file.write(html_summary_results)
html_file.close()